In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np


# Import API key, for some reason can import weather_api_key from config.py but wont import g_key so I just used the key and deleted.
g_key = 

# Configure gmaps API key
gmaps.configure(api_key = g_key)

In [2]:
# Import the WeatherPy_database.csv file. 
city_data_to_load = "C:/Users/lkand/Desktop/Weather_Database/Resources/WeatherPy_Database.csv"

city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Date
0,0,Qaanaaq,77.4840,-69.3632,38.26,80,97,8.66,GL,overcast clouds,2022-09-06 18:00:21
1,1,Cape Town,-33.9258,18.4232,57.27,65,0,13.80,ZA,clear sky,2022-09-06 17:59:44
2,2,Rikitea,-23.1203,-134.9692,73.51,69,99,22.44,PF,overcast clouds,2022-09-06 18:00:21
3,3,Castro,-24.7911,-50.0119,55.58,97,100,10.40,BR,overcast clouds,2022-09-06 18:00:22
4,4,Alta Floresta,-9.8756,-56.0861,98.42,30,68,5.75,BR,smoke,2022-09-06 17:59:57


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the coldest temperature you are ok with? "))
max_temp = float(input("What is the warmest temperature you are ok with? "))



What is the coldest temperature you are ok with? 31
What is the warmest temperature you are ok with? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Date
0,0,Qaanaaq,77.4840,-69.3632,38.26,80,97,8.66,GL,overcast clouds,2022-09-06 18:00:21
1,1,Cape Town,-33.9258,18.4232,57.27,65,0,13.80,ZA,clear sky,2022-09-06 17:59:44
2,2,Rikitea,-23.1203,-134.9692,73.51,69,99,22.44,PF,overcast clouds,2022-09-06 18:00:21
3,3,Castro,-24.7911,-50.0119,55.58,97,100,10.40,BR,overcast clouds,2022-09-06 18:00:22
5,5,Kapaa,22.0752,-159.3190,80.92,84,40,13.80,US,moderate rain,2022-09-06 18:00:22
6,6,Salalah,17.0151,54.0924,77.45,91,100,5.91,OM,overcast clouds,2022-09-06 18:00:22
7,7,Los Llanos De Aridane,28.6585,-17.9182,73.15,84,40,13.80,ES,scattered clouds,2022-09-06 18:00:23
8,8,Mahebourg,-20.4081,57.7000,71.26,64,61,14.63,MU,broken clouds,2022-09-06 18:00:23
9,9,Arraial Do Cabo,-22.9661,-42.0278,68.61,68,76,17.85,BR,broken clouds,2022-09-06 18:00:24
10,10,Jamestown,42.0970,-79.2353,69.06,66,100,8.05,US,overcast clouds,2022-09-06 18:00:24


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.isnull().sum()

639


City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                6
Weather Description    0
Date                   0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

633


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description,Date
0,0,Qaanaaq,77.4840,-69.3632,38.26,80,97,8.66,GL,overcast clouds,2022-09-06 18:00:21
1,1,Cape Town,-33.9258,18.4232,57.27,65,0,13.80,ZA,clear sky,2022-09-06 17:59:44
2,2,Rikitea,-23.1203,-134.9692,73.51,69,99,22.44,PF,overcast clouds,2022-09-06 18:00:21
3,3,Castro,-24.7911,-50.0119,55.58,97,100,10.40,BR,overcast clouds,2022-09-06 18:00:22
5,5,Kapaa,22.0752,-159.3190,80.92,84,40,13.80,US,moderate rain,2022-09-06 18:00:22


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Qaanaaq,GL,38.26,overcast clouds,77.4840,-69.3632,
1,Cape Town,ZA,57.27,clear sky,-33.9258,18.4232,
2,Rikitea,PF,73.51,overcast clouds,-23.1203,-134.9692,
3,Castro,BR,55.58,overcast clouds,-24.7911,-50.0119,
5,Kapaa,US,80.92,moderate rain,22.0752,-159.3190,
6,Salalah,OM,77.45,overcast clouds,17.0151,54.0924,
7,Los Llanos De Aridane,ES,73.15,scattered clouds,28.6585,-17.9182,
8,Mahebourg,MU,71.26,broken clouds,-20.4081,57.7000,
9,Arraial Do Cabo,BR,68.61,broken clouds,-22.9661,-42.0278,
10,Jamestown,US,69.06,overcast clouds,42.0970,-79.2353,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.loc[index,"Hotel Name"]=np.nan 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
hotel_df.count()


City                   633
Country                633
Max Temp               633
Weather Description    633
Lat                    633
Lng                    633
Hotel Name             568
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Qaanaaq,GL,38.26,overcast clouds,77.4840,-69.3632,Qaanaaq Hotel
1,Cape Town,ZA,57.27,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
2,Rikitea,PF,73.51,overcast clouds,-23.1203,-134.9692,People ThankYou
3,Castro,BR,55.58,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
5,Kapaa,US,80.92,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
687,Bereda,ES,69.42,overcast clouds,43.2686,-7.5406,Pazo de Verdes
689,Sungai Petani,MY,82.27,overcast clouds,5.6470,100.4877,Swiss Inn Sungai Petani
690,Pochutla,MX,87.82,overcast clouds,15.7432,-96.4661,Hotel Posada San Jose
691,Sitka,US,57.09,light rain,57.0531,-135.3300,Westmark Sitka


In [11]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))